In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import torch

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!pip install -q datasets
!pip install -q evaluate

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [ ]:
import evaluate

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
full_data_for_ranking = pd.read_excel('/content/drive/MyDrive/datasets/Budget_Speech_Sector_Company/sectorwise_budgetdaywise_performance_with_text_ranked_full.xlsx')
full_data_for_ranking

date          year budget_full_text_filename                 sector  \
0   1997-02-28     1997-1998              bs199798.txt                Finance   
1   1997-02-28     1997-1998              bs199798.txt               Textiles   
2   1998-01-06     1998-1999              bs199899.txt              Chemicals   
3   1998-01-06     1998-1999              bs199899.txt       Auto Ancillaries   
4   1998-03-25  1998-1999(I)           bs199899(I).txt       Auto Ancillaries   
..         ...           ...                       ...                    ...   
424 2025-01-02     2025-2026             bs2025_26.txt            Fertilizers   
425 2025-01-02     2025-2026             bs2025_26.txt  Air Transport Service   
426 2025-01-02     2025-2026             bs2025_26.txt         Agro Chemicals   
427 2025-01-02     2025-2026             bs2025_26.txt                Leather   
428 2025-01-02     2025-2026             bs2025_26.txt          IT - Hardware   

                                          text_segment  number_of_companies  \
0    ['Proposal to reduce corporate tax rates: Dome...                    2   
1    ['Reduction of duty on synthetic rubber, fibre...                    1   
2    ['The duty on photographic chemicals is being ...                    1   
3    ["Industrial deregulation would remain incompl...                    1   
4    ['industry continues to be in the doldrums ave...                    1   
..                                                 ...                  ...   
424  ['National Mission on High Yielding Seeds will...                   19   
425  ['Modified UDAN scheme to enhance regional con...                    2   
426  ['National Mission on High Yielding Seeds will...                   21   
427  ['Focus Product Scheme for Footwear & Leather ...                   12   
428  ['Reduction in BCD on Carrier-Grade Ethernet S...                    7   

     majority_target_binary  mean_target_binary  \
0                       1.0            1.000000   
1                       1.0            1.000000   
2                       1.0            1.000000   
3                       0.0            0.000000   
4                       0.0            0.000000   
..                      ...                 ...   
424                     1.0            0.526316   
425                     0.5            0.500000   
426                     0.0            0.476190   
427                     0.0            0.416667   
428                     0.0            0.285714   

     mean_target_percentage_increment  rank  
0                            0.139282     1  
1                            0.003745     2  
2                            0.026786     1  
3                           -0.101090     2  
4                           -0.033730     1  
..                                ...   ...  
424                          0.001219    21  
425                          0.005490    22  
426                          0.000646    23  
427                         -0.002770    24  
428                         -0.007205    25  

[429 rows x 10 columns]

In [ ]:
yr1 = 2020
yr2 = 2024

full_data_for_ranking['year_use'] = full_data_for_ranking['year'].apply(lambda x : int(str(x)[:4]))
full_data_for_ranking['text_segment'] = full_data_for_ranking['text_segment'].apply(lambda x : eval(x))
full_data_for_ranking['text_use'] = full_data_for_ranking.apply(lambda x : x.sector + ' [SEP] ' +' '.join(x.text_segment), axis = 1)

train = full_data_for_ranking[full_data_for_ranking['year_use']<yr1]
valid = full_data_for_ranking[(full_data_for_ranking['year_use']>=yr1) & (full_data_for_ranking['year_use']<yr2)]
test = full_data_for_ranking[full_data_for_ranking['year_use']>=yr2]
train.shape, valid.shape, test.shape

((318, 12), (41, 12), (70, 12))

In [ ]:
train.head()

date          year budget_full_text_filename            sector  \
0 1997-02-28     1997-1998              bs199798.txt           Finance   
1 1997-02-28     1997-1998              bs199798.txt          Textiles   
2 1998-01-06     1998-1999              bs199899.txt         Chemicals   
3 1998-01-06     1998-1999              bs199899.txt  Auto Ancillaries   
4 1998-03-25  1998-1999(I)           bs199899(I).txt  Auto Ancillaries   

                                        text_segment  number_of_companies  \
0  [Proposal to reduce corporate tax rates: Domes...                    2   
1  [Reduction of duty on synthetic rubber, fibres...                    1   
2  [The duty on photographic chemicals is being r...                    1   
3  [Industrial deregulation would remain incomple...                    1   
4  [industry continues to be in the doldrums aver...                    1   

   majority_target_binary  mean_target_binary  \
0                     1.0                 1.0   
1                     1.0                 1.0   
2                     1.0                 1.0   
3                     0.0                 0.0   
4                     0.0                 0.0   

   mean_target_percentage_increment  rank  year_use  \
0                          0.139282     1      1997   
1                          0.003745     2      1997   
2                          0.026786     1      1998   
3                         -0.101090     2      1998   
4                         -0.033730     1      1998   

                                            text_use  
0  Finance [SEP] Proposal to reduce corporate tax...  
1  Textiles [SEP] Reduction of duty on synthetic ...  
2  Chemicals [SEP] The duty on photographic chemi...  
3  Auto Ancillaries [SEP] Industrial deregulation...  
4  Auto Ancillaries [SEP] industry continues to b...

In [ ]:
train['label'] = train['mean_target_percentage_increment'].apply(lambda x : 0 if x < 0 else 1)
valid['label'] = valid['mean_target_percentage_increment'].apply(lambda x : 0 if x < 0 else 1)
test['label'] = test['mean_target_percentage_increment'].apply(lambda x : 0 if x < 0 else 1)

In [ ]:
train['label'].nunique()

2

In [ ]:
train['label'].value_counts()

label
0    194
1    124
Name: count, dtype: int64

In [ ]:
valid['label'].value_counts()

label
1    32
0     9
Name: count, dtype: int64

In [ ]:
test['label'].value_counts()

label
1    41
0    29
Name: count, dtype: int64

# BERT

In [ ]:
# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=train['label'].nunique())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256)


train_dataset = Dataset.from_pandas(train[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(valid[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)

Map:   0%|          | 0/318 [00:00<?, ? examples/s]

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# 5. Define metrics
metric = evaluate.load("f1")

#from sklearn.metrics import f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')

# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,  # Change to your validation set
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 7. Train and evaluate
print("Starting training...")
trainer.train()

print("Evaluating model...")
results = trainer.evaluate()
print(f"Final evaluation results: {results}")

test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test evaluation results: {test_results}")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting training...


Evaluating model...


Final evaluation results: {'eval_loss': 0.7507959008216858, 'eval_f1': 0.5195640892579139, 'eval_runtime': 0.6146, 'eval_samples_per_second': 66.71, 'eval_steps_per_second': 4.881, 'epoch': 3.0}
Test evaluation results: {'eval_loss': 0.7122550010681152, 'eval_f1': 0.45668449197860966, 'eval_runtime': 0.9629, 'eval_samples_per_second': 72.698, 'eval_steps_per_second': 5.193, 'epoch': 3.0}


In [ ]:
def predict_class(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Get the model's prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the confidence score (probability)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)



    return probabilities



In [ ]:
# Example usage
text_to_classify = test['text_use'].to_list()[0]
probabilities = predict_class(text_to_classify)

probabilities

tensor([[0.5934, 0.4066]], device='cuda:0')

In [ ]:
all_predictions = []
for text_to_classify in test['text_use'].to_list():
  probabilities = predict_class(text_to_classify)
  #predicted_classes = [1 if probabilities[0][i].item() > th else 0 for i in range(len(probabilities[0])) ]
  all_predictions.append(probabilities[0][1].item()) # Considering 0th element s it's a 2d array [[]]; Taking probaility of first class so [1]

In [ ]:
test['predicted_probability'] = all_predictions

In [ ]:
def rank_sectors_new_based_on_predictions(df):
    """Ranks sectors based on predictions"""

    ranked_df = df.copy()

    #Group data by date
    dates = ranked_df['date'].unique()
    ranked_list = []

    for date in dates:
      current_date_df = ranked_df[ranked_df['date'] == date]

      #Sort by mean_target_binary (descending) then mean_target_percentage_increment (descending).
      current_date_df = current_date_df.sort_values(['predicted_probability'], ascending=[False]).reset_index()
      #print(current_date_df['date'])
      #Assign ranks
      current_date_df['predicted_rank'] = range(1, len(current_date_df) + 1)
      ranked_list.append(current_date_df)
      #ranked_list.append(pd.DataFrame({'date':[date], 'sector':[current_date_df['sector'].to_list()]}))
      #print(pd.DataFrame({'date':[date], 'sector':[current_date_df['sector'].to_list()]}))

    ranked_df = pd.concat(ranked_list)
    return ranked_df

In [ ]:
test_predicted_ranked = rank_sectors_new_based_on_predictions(test)

In [ ]:
test_predicted_ranked

index       date          year budget_full_text_filename  \
0     359 2024-01-02  2024-2025(I)          bs2024_25(I).txt   
1     368 2024-01-02  2024-2025(I)          bs2024_25(I).txt   
2     370 2024-01-02  2024-2025(I)          bs2024_25(I).txt   
3     369 2024-01-02  2024-2025(I)          bs2024_25(I).txt   
4     372 2024-01-02  2024-2025(I)          bs2024_25(I).txt   
..    ...        ...           ...                       ...   
20    414 2025-01-02     2025-2026             bs2025_26.txt   
21    423 2025-01-02     2025-2026             bs2025_26.txt   
22    404 2025-01-02     2025-2026             bs2025_26.txt   
23    424 2025-01-02     2025-2026             bs2025_26.txt   
24    408 2025-01-02     2025-2026             bs2025_26.txt   

                                    sector  \
0                         Gas Distribution   
1          Power Generation & Distribution   
2                           Petrochemicals   
3                                     FMCG   
4   Capital Goods-Non Electrical Equipment   
..                                     ...   
20                           Miscellaneous   
21   Infrastructure Developers & Operators   
22                           Ship Building   
23                             Fertilizers   
24                                Shipping   

                                         text_segment  number_of_companies  \
0   [Phased mandatory blending of compressed bioga...                    8   
1   [Viability gap funding will be provided for ha...                   25   
2   [Viability gap funding will be provided for ha...                   11   
3   [Pradhan Mantri Kisan Sampada Yojana has benef...                   62   
4   [Viability gap funding will be provided for ha...                   48   
..                                                ...                  ...   
20  [Development of 50 tourist destinations with i...                   50   
21  [A Maritime Development Fund with a corpus of ...                   81   
22  [Shipbuilding Financial Assistance Policy will...                    1   
23  [National Mission on High Yielding Seeds will ...                   19   
24  [Shipbuilding Financial Assistance Policy will...                    4   

    majority_target_binary  mean_target_binary  \
0                      1.0            0.750000   
1                      0.0            0.400000   
2                      0.0            0.363636   
3                      0.0            0.387097   
4                      0.0            0.333333   
..                     ...                 ...   
20                     1.0            0.580000   
21                     1.0            0.530864   
22                     1.0            1.000000   
23                     1.0            0.526316   
24                     1.0            0.750000   

    mean_target_percentage_increment  rank  year_use  \
0                           0.014235     1      2024   
1                           0.008205    10      2024   
2                           0.009183    12      2024   
3                           0.000329    11      2024   
4                          -0.003791    14      2024   
..                               ...   ...       ...   
20                          0.007122    11      2025   
21                          0.003909    20      2025   
22                          0.008029     1      2025   
23                          0.001219    21      2025   
24                          0.003532     5      2025   

                                             text_use  label  \
0   Gas Distribution [SEP] Phased mandatory blendi...      1   
1   Power Generation & Distribution [SEP] Viabilit...      1   
2   Petrochemicals [SEP] Viability gap funding wil...      1   
3   FMCG [SEP] Pradhan Mantri Kisan Sampada Yojana...      1   
4   Capital Goods-Non Electrical Equipment [SEP] V...      0   
..                                                ...    ...   
20  Miscellaneous [SEP]

In [ ]:
import numpy as np

def softmax(x):
    """
    Compute softmax values for each set of scores in x.

    Parameters:
    x (array-like): Input array of scores.

    Returns:
    np.ndarray: Softmax probabilities.
    """
    exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exp_x / np.sum(exp_x)

# Example usage
scores = np.array(test['mean_target_percentage_increment'])
print("Softmax:", softmax(scores))


Softmax: [0.01446822 0.01454447 0.01437718 0.01440843 0.01425616 0.0143512
 0.01425832 0.01423136 0.01430161 0.01438124 0.01426842 0.01439532
 0.01446333 0.01420975 0.01426013 0.01424175 0.01411755 0.01422049
 0.01410303 0.01401228 0.01391271 0.01389454 0.01516016 0.01451173
 0.01428628 0.01450923 0.01433194 0.01429963 0.01423229 0.01431641
 0.0143507  0.0142468  0.01427842 0.01427247 0.01423645 0.0142463
 0.0142383  0.01417547 0.01420635 0.01413943 0.01418327 0.01421593
 0.01405583 0.01400254 0.01348911 0.0143787  0.01433833 0.01463129
 0.01447551 0.01431419 0.01434381 0.01437613 0.01437583 0.01423669
 0.01441176 0.01436567 0.01435922 0.01425909 0.01431577 0.01429905
 0.0142885  0.01429866 0.01433592 0.01433183 0.01431959 0.01428112
 0.01434225 0.01427294 0.01422427 0.01416132]


In [ ]:
test['mean_target_percentage_increment_transformed'] = list(softmax(scores))

In [ ]:
from sklearn.metrics import ndcg_score



ndcg_score_all = []

dates = test['date'].unique()


for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

In [ ]:
np.mean(ndcg_score_all)

0.9966306708574434

# RoBERTa

In [ ]:
# Load tokenizer and model
model_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=train['label'].nunique())

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256)


train_dataset = Dataset.from_pandas(train[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(valid[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# 5. Define metrics
metric = evaluate.load("f1")

#from sklearn.metrics import f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')


# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,  # Change to your validation set
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 7. Train and evaluate
print("Starting training...")
trainer.train()

print("Evaluating model...")
results = trainer.evaluate()
print(f"Final evaluation results: {results}")

test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test evaluation results: {test_results}")

def predict_class(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Get the model's prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the confidence score (probability)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)


    return probabilities


all_predictions = []
for text_to_classify in test['text_use'].to_list():
  probabilities = predict_class(text_to_classify)
  #predicted_classes = [1 if probabilities[0][i].item() > th else 0 for i in range(len(probabilities[0])) ]
  all_predictions.append(probabilities[0][1].item()) # Considering 0th element s it's a 2d array [[]]; Taking probaility of first class so [1]

test['predicted_probability'] = all_predictions

scores = np.array(test['mean_target_percentage_increment'])
test['mean_target_percentage_increment_transformed'] = list(softmax(scores))

from sklearn.metrics import ndcg_score



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting training...


Evaluating model...


Final evaluation results: {'eval_loss': 0.8517336845397949, 'eval_f1': 0.07902439024390244, 'eval_runtime': 0.6004, 'eval_samples_per_second': 68.288, 'eval_steps_per_second': 4.997, 'epoch': 3.0}
Test evaluation results: {'eval_loss': 0.7584280967712402, 'eval_f1': 0.2427128427128427, 'eval_runtime': 0.8459, 'eval_samples_per_second': 82.751, 'eval_steps_per_second': 5.911, 'epoch': 3.0}


0.9948203035865727

In [ ]:
ndcg_score_all = []

dates = test['date'].unique()


for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

In [ ]:
ndcg_score_all

[0.9943764062136131, 0.9909478171406789, 0.9970471346312632]

In [ ]:
np.mean(ndcg_score_all)

0.994123785995185

# DeBERTa

In [ ]:
# Load tokenizer and model
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=train['label'].nunique())

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256)


train_dataset = Dataset.from_pandas(train[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(valid[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test[['text_use', 'label']].rename(columns = {'text_use' : 'text'})).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# 5. Define metrics
metric = evaluate.load("f1")

#from sklearn.metrics import f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')


# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,  # Change to your validation set
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 7. Train and evaluate
print("Starting training...")
trainer.train()

print("Evaluating model...")
results = trainer.evaluate()
print(f"Final evaluation results: {results}")

test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test evaluation results: {test_results}")

def predict_class(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Get the model's prediction
    with torch.no_grad():
        outputs = model(**inputs)



    # Get the confidence score (probability)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)


    return probabilities


all_predictions = []
for text_to_classify in test['text_use'].to_list():
  probabilities = predict_class(text_to_classify)
  all_predictions.append(probabilities[0][1].item()) # Considering 0th element s it's a 2d array [[]]; Taking probaility of first class so [1]

test['predicted_probability'] = all_predictions

scores = np.array(test['mean_target_percentage_increment'])
test['mean_target_percentage_increment_transformed'] = list(softmax(scores))

from sklearn.metrics import ndcg_score

ndcg_score_all = []

dates = test['date'].unique()


for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

print(np.mean(ndcg_score_all))

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

...

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting training...


Evaluating model...


Final evaluation results: {'eval_loss': 0.802520751953125, 'eval_f1': 0.07902439024390244, 'eval_runtime': 0.8358, 'eval_samples_per_second': 49.056, 'eval_steps_per_second': 3.589, 'epoch': 3.0}
Test evaluation results: {'eval_loss': 0.7350812554359436, 'eval_f1': 0.2427128427128427, 'eval_runtime': 1.285, 'eval_samples_per_second': 54.474, 'eval_steps_per_second': 3.891, 'epoch': 3.0}
0.9964961156708267
